# Modern Data Science 
**(Module 02: Data Visualization)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au), Australia

---

Python Version = 2.7

# Session P - Dot Density Map

In our last lesson, we used the [basemap][1] library to create a choropleth showing the unemployment data for the US during the Great Recession. While choropleths are great for showing data that's associated with a specific geographic area, like a county for instance, when you want to take a more granular look at your data, you may find that a [dot density map][2] is the more appropriate tool for the job. In this lesson, we'll create a one-to-one dot density map of the crime in San Francisco using images from the the [ArcGIS online service][3]. 

[1]: http://matplotlib.org/basemap/
[2]: https://en.wikipedia.org/wiki/Dot_distribution_map
[3]: http://www.arcgis.com/features/

## What is a Dot Density Map?

While the name may sound intimidating, the actual concept is quite simple, in fact, I would wager that nearly all of us already quite familiar with the concept of a dot density map. A dot density map is what many of us would associate with the term heat map. But, while a heat map is a more generic term, a dot density map means something very specific. Like a choropleth, it maps the presence of a specific phenomenon, but where a choropleth associates a value with a specific geographic boundary, like a county or state, a dot density map, plots the actual location of the phenomenon's occurrence. Doing so allows the viewer to quickly see where areas of large observations of the phenomenon occur. 

As an example, let's take a look at one of the most historically important dot density maps: the English physician [John Snow][1]'s map of cholera cases during the [London epidemic of 1854][2]. 

<img src="https://upload.wikimedia.org/wikipedia/commons/2/27/Snow-cholera-map-1.jpg" alt="Cholera Map of 1854" width="800" />

We can see in the map above, that the number of cases is largest towards the center of the map. By plotting the actual occurrences themselves, rather than an aggregate for the area (as you would have with a choropleth), it was easy for Dr. Snow to see that the largest concentration of cholera cases occurred around the intersection of Broad St. and Little Windmill St., where a water pump for the town was located. This observation helped Dr. Snow conclude that contaminated water was the culprit in the spread of the illness, and ultimately led to the end of the epidemic. Had a map with a lower granularity, say a choropleth, been used, this discovery may have taken longer to come about, and many lives would have been unnecessarily lost in the interim.

So, from this historical example, it's easy to see just how useful a dot density map can be. So then, how do we make one ourselves? Well, we start by setting up our notebook and getting the data.

[1]: https://en.wikipedia.org/wiki/John_Snow_(physician)
[2]: https://en.wikipedia.org/wiki/1854_Broad_Street_cholera_outbreak

In [ ]:
!conda install basemap

In [ ]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

# Use the following only if you are on a high definition device
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

# The current version of NumPy available from conda is issuing a warning 
# message that some behavior will change in the future when used with the 
# current version of matplotlib available from conda. This cell just keeps
# that warning from being displayed.
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)


# Sure-fire way to turn off interactive mode in a Jupyter notebook

plt.ioff()
# Get a reference to the current IPython shell
shell = get_ipython()
# Import the event handler function that we are trying to unregister
from ipykernel.pylab.backend_inline import flush_figures
# Unregister the event handler for the current shell session
shell.events.unregister('post_execute', flush_figures)

## Getting the Data

Just like in our last lesson, in this lesson, we'll once again be using real world data, this time from the [data.sfgov.org][1] website.

So, the very first step is to click on the [link to the website][1] to open it up in a separate browser window. We'll be working with crime data, so you'll need to click on the [Public Safety][2] link, and then the link for [SFPD Incidents][3]. If you followed the links correctly, you should end up on a page that looks just like the image below.

![SFPD Incidents](./images/SFPD Incidents.png "SFPD Incidents")

[1]: https://data.sfgov.org 
[2]: https://data.sfgov.org/data?category=Public%20Safety
[3]: https://data.sfgov.org/Public-Safety/SFPD-Incidents-from-1-January-2003/tmnf-yvry

We're going to take a look at the crime in San Francisco over the past 6 months, so you'll need to select the beginning and end dates in the "Date is between" filter (you should notice the data being displayed automatically updates once you've selected your dates). Then, select the "Export" button at the top of the page and choose "JSON" as the download type. 

> The JSON file doesn't actually download to your machine, so you'll probably need to save the file once it's done loading.

Once, you get the file downloaded, you'll need to place it in a location where your code can find it. The code in this example assumes that the data can be found in the `data` directory for this notebook. And, to make things simpler on you, I've included the same data file that I'm using here in the `data` directory in the accompanying repository---so, if you're feeling lazy, you can actually skip everything that we've just done to get the data for this particular example.

One final note on getting the data, unless you change the name when you download the file, it'll be saved as `rows.json`, but that's not a terribly descriptive name, so I changed mine to `sfpd.json`. You can use either one, but again, remember if you're not using the data file that comes with the repository, you'll either need to change the name of the downloaded file, or alter the code below for it to work.

> Incidentally, if you are following along and would like to get the exact same results as I do here without using the data file in repository, you'll need use the same dates that I do. As of this recording, the past six month period would be 04/01/2015 - 10/01/2015.

So, you have the data, and now you just need to read it in. Easy enough, just open the file and load it in using the `json` module, like the code below shows.

In [ ]:
import wget

link_to_data = 'https://github.com/tuliplab/mds/blob/master/Jupyter/data/data_first%20step/sfpd.json'
DataSet = wget.download(link_to_data)

In [ ]:
!ls ./


In [ ]:
import json

with open('sfpd.json') as fin:
    data = json.load(fin);

We now have the data as a simple python dict, but we can do a little better than that. The data we got back from the website has two main attributes: a `data` object, which is just an array of incidents, and a `meta` object that contains, amongst other bits of information, the name of each field in an incident. We can use `namedtuple`s once again to combine the metadata with the incidents to create a list of objects that are much easier to work with.

The first step is to extract the names of the fields from the metadata. We'll use a list comprehension to loop through the column objects in the `meta.view.columns` list and extract the name for each field.

In [ ]:
fields = [obj['name'] for obj in data['meta']['view']['columns']]

Then, we'll import the `namedtuple` function and use it to create an `Incident` record with the list of fields passed in for the `field_names` parameter.

In [ ]:
from collections import namedtuple

Incident = namedtuple('Incident', fields)

Finally, we'll loop through the list of incidents in the `data` list and wrap each one in our `Incident` constructor, to create a list of incidents that'll be much easier to work with.

In [ ]:
incidents = [Incident(*row) for row in data['data']]    

## Creating the Basemap Object

To create our map object, we'll need a couple of things. First, we need to specify a projection to use. A map projection, in case you forgot, is a mechanism for translating points on a spherical surface, like the earth, onto a 2-dimensional plane, like a map (for a more in-depth explanation of this concept, please take a look at the first three lessons of this section).

So far we've only seen how to specify a projection by passing in its name to the `Basemap` constructor via the [`projection`][1] parameter. There is another way, however, the [`epsg`][2] argument. The former is a string representing one of the projections supported by the basemap library (incidentally, you can print the `basemap.supported_projections` variable for the full list of supported projections). An EPSG code is used to specify a coordinate reference system, which in layman's terms is just a way to turn points on the surface of the earth into x and y coordinates for use in a map. So, for our purposes, they can be considered the same thing as a map projection, and I'll be using the term projection from here on out to represent both. 

The projection we'll be using for this lesson is known as the [World Mercator][3] projection. It's used by nearly every online mapping application, and it's even referred to as the Google Projection for just that reason, and so it's a great choice for showing street level details. The EPSG code for the World Mercator projection is [(EPSG:3857)][4] and we'll pass that in with the `epsg` parameter when we create our `Basemap` instance.

The second piece of data we need is a bounding box for the portion of the map that we want to view. The Basemap constructor takes four different parameters that define this bounding box: two representing the upper right corner of the bounding box, and two more representing the lower left corner. And, just like you saw in the last lesson, you can easily get these coordinates using Google Maps to get the latitude and longitude for each corner.

To do so, simply open up a browser and navigate to [Google Maps][4]. Type 'San Francisco, CA' in the search box to center the map on San Francisco. Then, choose a point on the map that will serve as the upper right corner of the bounding box. This point will need to be far enough right so as to not cut off a portion of the city if a straight line were to be drawn through the point that was perpendicular to the bottom of the screen, and the same should be true for a line drawn through the point and perpendicular to the side of the screen. Right click on your chosen point and select "What's here?". A small rectangular box should pop up at the bottom of the screen containing, amongst other information, the latitude and longitude of the chosen point. This will be the lat/lon coordinates of the upper right corner of the bounding box. Now, do the same for the lower left corner and copy all of these coordinates into their corresponding parameters to create the Basemap object that we'll use throughout this lesson.


[1]: http://matplotlib.org/basemap/users/mapsetup.html
[2]: http://spatialreference.org
[3]: https://en.wikipedia.org/wiki/Web_Mercator
[4]: http://spatialreference.org/ref/sr-org/6864/
[5]: https://www.google.com/maps/

In [ ]:
map = Basemap(urcrnrlat=37.821981,     # top
              urcrnrlon=-122.343189,   # bottom
              llcrnrlat=37.697940,     # left
              llcrnrlon=-122.574915,   # right
              epsg=3857)

## Drawing the Map

Compared to the last few lessons, we'll be doing things a bit differently this time. In previous lessons, we drew the map ourselves using some built-in methods from the `Basemap` class, or by drawing the shapes ourselves, like we did for the county boundaries in the last lesson. In this lesson, however, we'll be using the `arcgisimage` method to retrieve a map image using the ArcGIS REST API. By default, the `arcgisimage` method pulls maps from the [ArcGIS online][1] service, but you can override that to pull images from any other service that implements the ArcGIS REST API, and we'll see an example of how to do that a little later on. 

Before we grab our map though, we'll need to adjust the size of the figure, otherwise we'll end up with something too small to be of any real use. The dimensions below are the width and height of the image we're going to pull from the server in inches. They may seem a bit large, but after playing around with the size a bit, I found that anything below a width of 20 inches would result in a map that did not include the smaller streets in the city. Not to worry about the size though, the notebook will resize the image to fit the cell, so you'll be able to see everything without scrolling all over the place.

[1]: http://server.arcgisonline.com/ArcGIS/rest/services

In [ ]:
# These are the dimensions of the map we'll create
width = 20
height = 10

fig = plt.gcf()
fig.set_size_inches(width, height);

We're also going to specify the pixel density of the image by setting the number of pixels on the `x-axis`, since the default density of 400 pixels is little low and lacking in detail. The default dpi for the image is 96, so we'll use that, along with the image dimensions that we want, to come up with a number for the `xpixels` parameter.

In [ ]:
# The default DPI setting is 96, we're just specifying it here 
# to use it for calculating the number of x-axis pixels to request.
dpi = 96
xpixels = dpi * width

Now, we'll need to pick the type of map that we want to view. Fortunately, it's really easy to do so, just follow the [ArcGIS services link][1], and you'll see a list of all the maps that the service provides. Select the one you want, and pop its name into the `service` parameter when you call the `arcgisimage` method. For this example, I've decided to go with the 'World_light_Gray_Base' map in the 'Canvas' folder, since the light gray background really makes the data stand out. 

[1]: http://server.arcgisonline.com/ArcGIS/rest/services

In [ ]:
map.arcgisimage(service='Canvas/World_Light_Gray_Base', xpixels=xpixels)
plt.show();

## Adding in the Data

Now that we've downloaded our data, chosen a map projection, and picked out a map image, we're ready to put everything together into a single map, right? Not so fast...

The data we download is real world data, i.e., it contains lat/lon coordinates on the earth. Before we can plot this data, we'll need to transform it into 2-dimensional x and y coordinates. So, let's do that now.

The first step in this process, is to extract the latitude and longitude from each of our incident objects. The code below creates two lists: one containing all of the latitude coordinates in our data, and the other containing the longitude.

In [ ]:
lon = [float(i.X) for i in incidents]
lat = [float(i.Y) for i in incidents]

The next steps is to transform our longitude and latitude lists into lists of x and y coordinates respectively. One of the nice features of a `Basemap` object is that it's a callable object that takes lists of geographic coordinates and returns lists of cartesian coordinates. So, all we need to do now is call the map projection we created earlier with the lists from the previous step, and we'll have the data in the format we need to plot on our map.

On a side note, calling a `Basemap` instance with the `inverse` parameter set to `True` will reverse this transformation we're about to perform.

In [ ]:
x, y = map(lon, lat)

Now, we have our map and we have our data in the correct format, the only step left is to combine the two. Just like we did above, we'll first set the dimensions of the map. Then, we'll draw the map. Finally, we'll call the `scatter` method on the `map` object and pass in the x and y coordinate lists we created in the last step. In this step, we're also going to change a few aesthetic features as well. 

First, we'll change the color of the markers to red and set the `alpha` (the opacity of each marker) to just 1%. The alpha setting is crucial since it allows the map to show different concentrations of crime with more or less intensity. Without this setting any valuable information we could gleam from the map would be lost in a sea of red. 

Then, we'll change the shape of the markers to be square, and the size of each to be slightly smaller than the default. 

Finally, we'll remove the outline from each marker by setting the `linewidths` parameter to 0.

In [ ]:
plt.gcf().set_size_inches(width, height)
map.arcgisimage(service='Canvas/World_Light_Gray_Base', xpixels=xpixels)
map.scatter(x, y, c='r', alpha=0.01, marker='s', s=15, linewidths=0)
plt.show()

## A Few More Examples

Before we bring this lesson to a close, we're going to take a look at two more examples of our dot density map. The first example below, simply uses a different service, i.e., a different map, from the ArcGIS servers. 

In this example, we'll show our data overlaid atop a satellite image of the city of San Francisco.

In [ ]:
plt.gcf().set_size_inches(width, height)
map.arcgisimage(service='ESRI_Imagery_World_2D', xpixels=xpixels)
map.scatter(x, y, c='r', alpha=0.01, marker='s', s=15, linewidths=0)
plt.show()

## Using a Different Server

Finally, as I mentioned earlier, by default the `arcgisimage` method uses the [ArcGIS online service][1], but you could override this default setting to use any service that implements the ArcGIS REST API. 

In the example below, we go with a topological map of San Francisco provided by the [U.S. Geological Survey (USGS)][2]. To do so, we simply need to change the default service by setting the `server` parameter to [USGS's ArcGIS service][3]. Outside of that one change, everything else is identical to the calls we made above.

[1]: http://server.arcgisonline.com/ArcGIS/rest/services
[2]: http://www.usgs.gov
[3]: http://basemap.nationalmap.gov/arcgis/rest/services

In [ ]:
plt.gcf().set_size_inches(width, height)
map.arcgisimage(server='http://basemap.nationalmap.gov/arcgis',
                service='USGSTopo', 
                xpixels=xpixels)
map.scatter(x, y, c='r', alpha=0.01, marker='s', s=15, linewidths=0)
plt.show()

## Conclusion

And, that will bring us to the end of this lesson, and to the end of this section on mapping data with the basemap library. 

In this section, we've learned the basics of the basemap library, as well as some theory behind the issues around creating maps. And, in the past two lessons, we've seen examples of how we can use this tool to present useful visualizations of real world geographical data. Hopefully, this section, and the last two lessons in particular, will inspire you to grab some interesting data, and map it!